In [1]:
%load_ext autoreload
%autoreload 2

# IMPORTS

import tensorflow as tf
from tensorflow.keras.datasets import cifar10

import tf2cv, cv2, PIL, os
from tf2cv.model_provider import get_model
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display, Image
import time
import warnings
warnings.filterwarnings('ignore')


# SETTINGS

CLASSES = ["airplane", "automobile", "bird", "cat","deer","dog","frog","horse","ship","truck"]

GPUs = [d for d in tf.config.list_physical_devices() if "GPU" in d.device_type]
DEVICE = tf.device("GPU") if (len(GPUs) > 0) else tf.device("CPU")

DTYPE = "float32" # float64

tf.keras.backend.set_floatx(DTYPE) # sets network layers to DTYPE

BS = 64 # autoencoder batch size
DIM = 32 # pixel dimensions, e.g. 32 for 32x32 color images


# LOAD DATASET

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

xtrain = tf.convert_to_tensor(x_train.astype(DTYPE) / 255.)
xtest = tf.convert_to_tensor(x_test.astype(DTYPE) / 255.)
ytrain = tf.convert_to_tensor(y_train.flatten())
ytest = tf.convert_to_tensor(y_test.flatten())


def augment(imgs):
    imgs = tf.image.rot90(imgs, np.random.randint(0,4))
    imgs = tf.image.random_flip_left_right(imgs)
    imgs = tf.image.random_flip_up_down(imgs)
    imgs = tf.image.random_hue(imgs, 0.07)
    imgs = tf.image.random_saturation(imgs, 0.7, 1.5)
    imgs = tf.image.random_brightness(imgs, 0.05)
    imgs = tf.image.random_contrast(imgs, 0.6, 1.2)
    imgs = tf.clip_by_value(imgs, 0, 1)
    return imgs

def preprocess(imgs):
    "Normalizes images (according to https://github.com/osmr/imgclsmob/blob/master/examples/demo_tf2.py)"
    mean_rgb = tf.constant([0.485, 0.456, 0.406]) # CIFAR-10 RGB mean
    std_rgb = tf.constant([0.229, 0.224, 0.225]) # CIFAR-10 RGB standard deviation
    mean_rgb = tf.cast(mean_rgb, DTYPE)
    std_rgb = tf.cast(std_rgb, DTYPE)
    return (imgs - mean_rgb)/std_rgb
#     return tf.clip_by_value((imgs - mean_rgb) / std_rgb, 0, 1)

def undo_preprocess(imgs):
    "restores preprocessed images to original state"
    mean_rgb = tf.constant([0.485, 0.456, 0.406]) # CIFAR-10 RGB mean
    std_rgb = tf.constant([0.229, 0.224, 0.225]) # CIFAR-10 RGB standard deviation
    mean_rgb = tf.cast(mean_rgb, DTYPE)
    std_rgb = tf.cast(std_rgb, DTYPE)
    return imgs*std_rgb + mean_rgb
#     return tf.clip_by_value(imgs*std_rgb + mean_rgb, 0, 1)

def evaluate(model, imgs, labels, loss_fn=tf.keras.losses.CategoricalCrossentropy(), preproc=True):
    "Returns loss and predictions"
    if preproc:
        imgs = preprocess(imgs)
    preds = tf.nn.softmax(model(imgs))
    if labels.shape[-1] != len(CLASSES):
        final_preds = tf.argmax(preds, axis=1)
        final_preds = tf.cast(final_preds, labels.dtype)
        labels = tf.one_hot(labels, depth=len(CLASSES), axis=1)
    loss = loss_fn(labels, preds)
    return loss, final_preds

def perturb(imgs, labels, model, loss_fn=tf.keras.losses.CategoricalCrossentropy()):
    "creates a perturbation (as a np.array) for a given image, label, and model"
    with tf.GradientTape() as g:
        g.watch(imgs)
        preds = tf.nn.softmax(model(imgs))
        loss = loss_fn(labels, preds)
        grad = g.gradient(loss, imgs)
        grad = grad if grad is not None else tf.zeros(imgs.shape)
    return tf.sign(grad)

class Attack:
    def __init__(self, model, loss_fn=None, epsilon=8, target=None, preproc=False):
        """
        model: tf model
        loss_fn: tf loss function or None (default: CategoricalCrossentropy)
        epsilon: maximum image noise perturbation within [0,255]
        target: int (targeted) or None (untargeted)
        """
        assert type(target) == int or target == None
        assert type(loss_fn) != str
        assert epsilon < 255 and epsilon >= 1
        
        self.epsilon = epsilon / 255.
        self.model = model
        self.loss_fn = loss_fn if loss_fn is not None else tf.keras.losses.CategoricalCrossentropy()
        self.target = target
        self.preproc = preproc

    def FGSM(self, imgs, labels):
        "FGSM attack; inputs: tf tensors"
        if self.preproc:
            imgs = preprocess(imgs)
        if type(self.target) == int:
            labels = tf.constant([self.target]*imgs.shape[0])
        if labels.shape[-1] != len(CLASSES):
            labels = tf.one_hot(labels, depth=len(CLASSES), axis=1)

        noise = perturb(imgs, labels, self.model, self.loss_fn)
        noise = tf.clip_by_value(noise, -1, 1)
        adv_imgs = (imgs + self.epsilon*noise) if self.target is None else (imgs - self.epsilon*noise)
        adv_imgs = tf.clip_by_value(adv_imgs, 0, 1)
        if self.preproc:
            adv_imgs = undo_preprocess(adv_imgs)
        return adv_imgs
    
    def PGD(self, imgs, labels, learning_rate=0.01, steps=10, random_init=True):
        "PGD attack; inputs: tf tensors"
        if learning_rate >= 1 and learning_rate < 255:
            learning_rate = learning_rate / 255.
        if type(self.target) == int:
            labels = tf.constant([self.target]*imgs.shape[0])
        if labels.shape[-1] != len(CLASSES):
            labels = tf.one_hot(labels, depth=len(CLASSES), axis=1)

        if self.preproc:
            imgs = preprocess(imgs)
        x = imgs
        if random_init:
            init_noise = tf.random.uniform(imgs.shape, -self.epsilon, self.epsilon)
            init_noise = tf.cast(init_noise, x.dtype)
            x += init_noise
            x = tf.clip_by_value(x, 0, 1)

        for i in range(steps):
            noise = perturb(x, labels, self.model, self.loss_fn)
            noise = tf.clip_by_value(noise, -1, 1)

            if type(self.target) == int:
                x -= learning_rate * noise
            else:
                x += learning_rate * noise
            x = tf.clip_by_value(x, imgs - self.epsilon, imgs + self.epsilon)
            x = tf.clip_by_value(x, 0, 1)
        if self.preproc:
            x = undo_preprocess(x)
        return x
    
class Autoencoder(tf.keras.models.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(DIM, DIM, 3)),
            tf.keras.layers.Conv2D(int(DIM/2), kernel_size=3, activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2D(int(DIM/4), kernel_size=3, activation='relu', padding='same', strides=2),
        ])

        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(int(DIM/4), kernel_size=3, activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2DTranspose(int(DIM/2), kernel_size=3, activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2D(3, kernel_size=3, activation='sigmoid', padding='same')
        ])

    def call(self, x):
        return self.decoder(self.encoder(x))
    
    def summary(self, line_length=None, positions=None, print_fn=None):
        self.encoder.summary(line_length, positions, print_fn)
        self.decoder.summary(line_length, positions, print_fn)


In [2]:
model_names = [
    'resnet1202_cifar10',
    'preresnet1202_cifar10',
    'resnext272_1x64d_cifar10',
    'resnext272_2x32d_cifar10',
    'seresnet542bn_cifar10',
    'sepreresnet542bn_cifar10',
    'pyramidnet236_a220_bn_cifar10',
    'pyramidnet272_a200_bn_cifar10',
    'densenet250_k24_bc_cifar10'
]

In [3]:
model_names = [
    'resnet20_cifar10',
    'resnet56_cifar10',
    'resnet110_cifar10',
    'resnet164bn_cifar10',
    'resnet272bn_cifar10',
    'resnet542bn_cifar10',
    'resnet1202_cifar10',
    'preresnet20_cifar10',
    'preresnet56_cifar10',
    'preresnet110_cifar10',
    'preresnet164bn_cifar10',
    'preresnet272bn_cifar10',
    'preresnet542bn_cifar10',
    'preresnet1202_cifar10',
    'resnext20_1x64d_cifar10',
    'resnext20_2x32d_cifar10',
    'resnext20_2x64d_cifar10',
    'resnext20_4x16d_cifar10',
    'resnext20_4x32d_cifar10',
    'resnext20_8x8d_cifar10',
    'resnext20_8x16d_cifar10',
    'resnext20_16x4d_cifar10',
    'resnext20_16x8d_cifar10',
    'resnext20_32x2d_cifar10',
    'resnext20_32x4d_cifar10',
    'resnext20_64x1d_cifar10',
    'resnext20_64x2d_cifar10',
    'resnext29_32x4d_cifar10',
    'resnext29_16x64d_cifar10',
    'resnext56_1x64d_cifar10',
    'resnext56_2x32d_cifar10',
    'resnext56_4x16d_cifar10',
    'resnext56_8x8d_cifar10',
    'resnext56_16x4d_cifar10',
    'resnext56_32x2d_cifar10',
    'resnext56_64x1d_cifar10',
    'resnext272_1x64d_cifar10',
    'resnext272_2x32d_cifar10',
    'seresnet20_cifar10',
    'seresnet56_cifar10',
    'seresnet110_cifar10',
    'seresnet164bn_cifar10',
    'seresnet272bn_cifar10',
    'seresnet542bn_cifar10',
    'sepreresnet20_cifar10',
    'sepreresnet56_cifar10',
    'sepreresnet110_cifar10',
    'sepreresnet164bn_cifar10',
    'sepreresnet272bn_cifar10',
    'sepreresnet542bn_cifar10',
    'pyramidnet110_a48_cifar10',
    'pyramidnet110_a84_cifar10',
    'pyramidnet110_a270_cifar10',
    'pyramidnet164_a270_bn_cifar10',
    'pyramidnet200_a240_bn_cifar10',
    'pyramidnet236_a220_bn_cifar10',
    'pyramidnet272_a200_bn_cifar10',
]

In [4]:
model_names = [
    'resnet20_cifar10',
    'resnet56_cifar10',
    'resnet110_cifar10',
    'preresnet20_cifar10',
    'preresnet56_cifar10',
    'preresnet110_cifar10',
    'seresnet20_cifar10',
    'seresnet56_cifar10',
    'seresnet110_cifar10',
    'sepreresnet20_cifar10',
    'sepreresnet56_cifar10',
    'sepreresnet110_cifar10',
    'resnext20_32x2d_cifar10',
    'resnext20_32x4d_cifar10',
    'resnext20_64x1d_cifar10',
    'resnext20_64x2d_cifar10',
    'resnext29_32x4d_cifar10',
    'resnext29_16x64d_cifar10',
    'resnext56_1x64d_cifar10',
    'resnext56_2x32d_cifar10',
    'resnext56_4x16d_cifar10',
    'resnext56_8x8d_cifar10',
]

    0  resnet20_cifar10     0.7909     5.602394104003906
    1  resnet56_cifar10     0.7909     15.917900562286377
    2  resnet110_cifar10     0.8033     31.530543088912964
    3  preresnet20_cifar10     0.7979     5.553314208984375
    4  preresnet56_cifar10     0.8159     15.875879049301147
    5  preresnet110_cifar10     0.8242     31.465455055236816
    6  seresnet20_cifar10     0.7788     6.165999889373779
    7  seresnet56_cifar10     0.8041     17.61270236968994
    8  seresnet110_cifar10     0.8145     34.87532997131348
    9  sepreresnet20_cifar10     0.7675     6.213705778121948
    10  sepreresnet56_cifar10     0.8213     17.639922618865967
    11  sepreresnet110_cifar10     0.7995     34.84875535964966
    12  pyramidnet110_a48_cifar10     0.7534     52.499168395996094
    13  pyramidnet110_a84_cifar10     0.8044     75.8461856842041


In [5]:
model_names = [
    'preresnet110_cifar10',
    'sepreresnet56_cifar10',
    'sepreresnet110_cifar10',
    'preresnet56_cifar10',
    'resnet110_cifar10',
#     'preresnet164bn_cifar10',
#     'preresnet272bn_cifar10',
#     'preresnet542bn_cifar10',
#     'sepreresnet164bn_cifar10',
#     'sepreresnet272bn_cifar10',
#     'sepreresnet542bn_cifar10',
#     'resnext20_32x2d_cifar10',
#     'resnext20_32x4d_cifar10',
#     'resnext20_64x1d_cifar10',
#     'resnext20_64x2d_cifar10',
#     'resnext29_32x4d_cifar10',
#     'resnext29_16x64d_cifar10',
#     'resnext56_1x64d_cifar10',
#     'resnext56_2x32d_cifar10',
#     'resnext56_4x16d_cifar10',
#     'resnext56_8x8d_cifar10',
]

In [5]:
model_names = ["sepreresnet56_cifar10", "resnet110_cifar10"] # winners

In [6]:
t = time.time()
models = []
for name in model_names:
    print(name)
    model = get_model(name, pretrained=True)
    model.trainable = False
    models.append(model)
print("Time:", time.time()-t)

sepreresnet56_cifar10
resnet110_cifar10
Time: 7.314396858215332


In [7]:
models[0].summary()

Model: "cifarse_pre_res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (Sequential)        (1, 1, 1, 64)             865919    
_________________________________________________________________
output1 (Dense)              multiple                  650       
Total params: 866,569
Trainable params: 0
Non-trainable params: 866,569
_________________________________________________________________


In [8]:
models[1].summary()

Model: "cifar_res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (Sequential)        (1, 1, 1, 64)             1738352   
_________________________________________________________________
output1 (Dense)              multiple                  650       
Total params: 1,739,002
Trainable params: 0
Non-trainable params: 1,739,002
_________________________________________________________________


# Load Progress

In [7]:
# FOR AUTOENCODER TRAINING

fgsm_train_0 = np.load("fgsm_train_0.npy")
pgd_train_0 = np.load("pgd_train_0.npy")

fgsm_train_1 = np.load("fgsm_train_1.npy")
pgd_train_1 = np.load("pgd_train_1.npy")

fgsm_train_2 = np.load("fgsm_train_2.npy")
pgd_train_2 = np.load("pgd_train_2.npy")

fgsm_train_3 = np.load("fgsm_train_3.npy")
pgd_train_3 = np.load("pgd_train_3.npy")

fgsm_train_4 = np.load("fgsm_train_4.npy")
pgd_train_4 = np.load("pgd_train_4.npy")

fgsm_train = np.vstack([fgsm_train_0, fgsm_train_1, fgsm_train_2, fgsm_train_3, fgsm_train_4])
pgd_train = np.vstack([pgd_train_0, pgd_train_1, pgd_train_2, pgd_train_3, pgd_train_4])

fgsm_test = np.load("fgsm_test.npy")
pgd_test = np.load("pgd_test.npy")

all_train = np.vstack([xtrain, fgsm_train, pgd_train])
all_train_target = np.vstack([xtrain, xtrain, xtrain])

fgsm_train = tf.convert_to_tensor(fgsm_train.astype(DTYPE))
pgd_train = tf.convert_to_tensor(pgd_train.astype(DTYPE))
fgsm_test = tf.convert_to_tensor(fgsm_test.astype(DTYPE))
pgd_test = tf.convert_to_tensor(pgd_test.astype(DTYPE))

all_train = tf.convert_to_tensor(all_train.astype(DTYPE))
all_train_target = tf.convert_to_tensor(all_train_target.astype(DTYPE))

In [8]:
# FOR RESNET FINE TUNING

fgsm_train_dec_0 = np.load("fgsm_train_dec_0.npy")
pgd_train_dec_0 = np.load("pgd_train_dec_0.npy")
orig_train_dec_0 = np.load("orig_train_dec_0.npy")

fgsm_train_dec_1 = np.load("fgsm_train_dec_1.npy")
pgd_train_dec_1 = np.load("pgd_train_dec_1.npy")
orig_train_dec_1 = np.load("orig_train_dec_1.npy")

fgsm_train_dec_2 = np.load("fgsm_train_dec_2.npy")
pgd_train_dec_2 = np.load("pgd_train_dec_2.npy")
orig_train_dec_2 = np.load("orig_train_dec_2.npy")

fgsm_train_dec_3 = np.load("fgsm_train_dec_3.npy")
pgd_train_dec_3 = np.load("pgd_train_dec_3.npy")
orig_train_dec_3 = np.load("orig_train_dec_3.npy")

fgsm_train_dec_4 = np.load("fgsm_train_dec_4.npy")
pgd_train_dec_4 = np.load("pgd_train_dec_4.npy")
orig_train_dec_4 = np.load("orig_train_dec_4.npy")

fgsm_train_dec = np.vstack([fgsm_train_dec_0, fgsm_train_dec_1, fgsm_train_dec_2, fgsm_train_dec_3, fgsm_train_dec_4])
pgd_train_dec = np.vstack([pgd_train_dec_0, pgd_train_dec_1, pgd_train_dec_2, pgd_train_dec_3, pgd_train_dec_4])
orig_train_dec = np.vstack([orig_train_dec_0, orig_train_dec_1, orig_train_dec_2, orig_train_dec_3, orig_train_dec_4])

dec_orig_test = np.load("dec_orig_test.npy")
dec_fgsm_test = np.load("dec_fgsm_test.npy")
dec_pgd_test = np.load("dec_pgd_test.npy")

all_train = np.vstack([xtrain, fgsm_train, pgd_train, fgsm_train_dec, pgd_train_dec])
all_train_target = np.hstack([ytrain, ytrain, ytrain, ytrain, ytrain])

fgsm_train_dec = tf.convert_to_tensor(fgsm_train_dec.astype(DTYPE))
pgd_train_dec = tf.convert_to_tensor(pgd_train_dec.astype(DTYPE))
dec_orig_test = tf.convert_to_tensor(dec_orig_test.astype(DTYPE))
dec_fgsm_test = tf.convert_to_tensor(dec_fgsm_test.astype(DTYPE))
dec_pgd_test = tf.convert_to_tensor(dec_pgd_test.astype(DTYPE))

all_train = tf.convert_to_tensor(all_train.astype(DTYPE))
all_train_target = tf.convert_to_tensor(all_train_target.astype(DTYPE))

In [9]:
# FINE TUNE (AFTER 3 EPOCHS)

all_train = np.vstack([xtrain, fgsm_train_dec, pgd_train_dec])
all_train_target = np.hstack([ytrain, ytrain, ytrain])


all_train = tf.convert_to_tensor(all_train.astype(DTYPE))
all_train_target = tf.convert_to_tensor(all_train_target.astype(DTYPE))

In [10]:
# # FINE TUNE (AFTER 3+1+2 EPOCHS)

all_train = np.vstack([xtrain, orig_train_dec, fgsm_train_dec, pgd_train_dec])
all_train_target = np.hstack([ytrain, ytrain, ytrain, ytrain])

all_train = tf.convert_to_tensor(all_train.astype(DTYPE))
all_train_target = tf.convert_to_tensor(all_train_target.astype(DTYPE))

In [11]:
autoencoder = tf.keras.models.load_model("models/autoencoder_protection")
autoencoder.trainable = False
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 16)        448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 8)           1160      
Total params: 1,608
Trainable params: 0
Non-trainable params: 1,608
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 16, 16, 8)         584       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 16)        1168      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 3)         435       
Total params: 2,187
Trainable pa

In [12]:
# CHECK EFFECTIVENESS

# xtest = augment(xtest)
# fgsm_test = augment(fgsm_test)
# pgd_test = augment(pgd_test)

t = time.time()
enc_orig = autoencoder.encoder(xtest)
dec_orig = autoencoder.decoder(enc_orig)
t = time.time() - t
print(f"AE time: {t:2f}\n")

enc_fgsm = autoencoder.encoder(fgsm_test)
dec_fgsm = autoencoder.decoder(enc_fgsm)

enc_pgd = autoencoder.encoder(pgd_test)
dec_pgd = autoencoder.decoder(enc_pgd)

i = 0
for name, model in zip(model_names, models):
    
    loss, preds = evaluate(model, xtest, ytest)
    orig = np.mean(preds == ytest)
    t = time.time()
    loss, preds = evaluate(model, dec_orig, ytest)
    new = np.mean(preds == ytest)
    t = time.time() - t
    print(f"{i}  {name}    Original: {orig} --> {new}   (time: {t:.2f})")
    
    loss, preds = evaluate(model, fgsm_test, ytest)
    orig = np.mean(preds == ytest)
    t = time.time()
    loss, preds = evaluate(model, dec_fgsm, ytest)
    new = np.mean(preds == ytest)
    t = time.time() - t
    print(f"{i}  {name}    FGSM: {orig} --> {new}       (time: {t:.2f})")
    
    loss, preds = evaluate(model, pgd_test, ytest)
    orig = np.mean(preds == ytest)
    t = time.time()
    loss, preds = evaluate(model, dec_pgd, ytest)
    new = np.mean(preds == ytest)
    t = time.time() - t
    print(f"{i}  {name}    PGD: {orig} --> {new}        (time: {t:.2f})")
    i+=1

AE time: 0.640149

0  preresnet110_cifar10    Original: 0.9598 --> 0.8242   (time: 31.29)
0  preresnet110_cifar10    FGSM: 0.4774 --> 0.749       (time: 31.36)
0  preresnet110_cifar10    PGD: 0.3578 --> 0.7843        (time: 31.42)
1  sepreresnet56_cifar10    Original: 0.9532 --> 0.8213   (time: 17.45)
1  sepreresnet56_cifar10    FGSM: 0.4711 --> 0.7411       (time: 17.45)
1  sepreresnet56_cifar10    PGD: 0.2966 --> 0.7835        (time: 17.47)
2  sepreresnet110_cifar10    Original: 0.9535 --> 0.7995   (time: 34.39)
2  sepreresnet110_cifar10    FGSM: 0.5423 --> 0.7227       (time: 34.42)
2  sepreresnet110_cifar10    PGD: 0.3987 --> 0.7615        (time: 34.42)
3  preresnet56_cifar10    Original: 0.9543 --> 0.8159   (time: 15.93)
3  preresnet56_cifar10    FGSM: 0.4309 --> 0.7363       (time: 15.92)
3  preresnet56_cifar10    PGD: 0.3172 --> 0.7761        (time: 15.91)
4  resnet110_cifar10    Original: 0.9625 --> 0.8033   (time: 31.72)
4  resnet110_cifar10    FGSM: 0.4526 --> 0.724       (ti

Backup model: Use sepreresnet56_cifar10

    0  resnet20_cifar10     0.7909     5.602394104003906
    1  resnet56_cifar10     0.7909     15.917900562286377
    2  resnet110_cifar10     0.8033     31.530543088912964
    3  preresnet20_cifar10     0.7979     5.553314208984375
    4  preresnet56_cifar10     0.8159     15.875879049301147
    5  preresnet110_cifar10     0.8242     31.465455055236816
    6  seresnet20_cifar10     0.7788     6.165999889373779
    7  seresnet56_cifar10     0.8041     17.61270236968994
    8  seresnet110_cifar10     0.8145     34.87532997131348
    9  sepreresnet20_cifar10     0.7675     6.213705778121948
    10  sepreresnet56_cifar10     0.8213     17.639922618865967
    11  sepreresnet110_cifar10     0.7995     34.84875535964966
    12  pyramidnet110_a48_cifar10     0.7534     52.499168395996094
    13  pyramidnet110_a84_cifar10     0.8044     75.8461856842041


EfficientNet architecture:

    1 - Stem    - Conv3x3|BN|Swish

    2 - Blocks  - MBConv1, k3x3 
                - MBConv6, k3x3 repeated 2 times
                - MBConv6, k5x5 repeated 2 times
                - MBConv6, k3x3 repeated 3 times
                - MBConv6, k5x5 repeated 3 times
                - MBConv6, k5x5 repeated 4 times
                - MBConv6, k3x3
                                totally 16 blocks

    3 - Head    - Conv1x1|BN|Swish 
                - Pooling
                - Dropout
                - FC
                
        where Swish(x) = x * sigmoid(x)



https://www.tensorflow.org/api_docs/python/tf/keras/applications/EfficientNetB6

In [18]:
# CHECK EFFECTIVENESS

# xtest = augment(xtest)
# fgsm_test = augment(fgsm_test)
# pgd_test = augment(pgd_test)

enc_orig = autoencoder.encoder(xtest)
dec_orig = autoencoder.decoder(enc_orig)

enc_fgsm = autoencoder.encoder(fgsm_test)
dec_fgsm = autoencoder.decoder(enc_fgsm)

enc_pgd = autoencoder.encoder(pgd_test)
dec_pgd = autoencoder.decoder(enc_pgd)

for name, model in zip(model_names, models):

    t = time.time()
    loss, preds = evaluate(model, xtest, ytest)
    orig = np.mean(preds == ytest)
    print(f"{name} - time: ",time.time() - t,"\n")
    loss, preds = evaluate(model, dec_orig, ytest)
    new = np.mean(preds == ytest)
    print("  (Test Accuracy) With autoencoder protection:")
    print(f"    Regular: {orig} --> {new}")

    loss, preds = evaluate(model, fgsm_test, ytest)
    orig = np.mean(preds == ytest)
    loss, preds = evaluate(model, dec_fgsm, ytest)
    new = np.mean(preds == ytest)
    print(f"  FGSM: {orig} --> {new}")

    loss, preds = evaluate(model, pgd_test, ytest)
    orig = np.mean(preds == ytest)
    loss, preds = evaluate(model, dec_pgd, ytest)
    new = np.mean(preds == ytest)
    print(f"  PGD: {orig} --> {new}\n")

resnet20_cifar10 - time:  5.414407014846802 

  (Test Accuracy) With autoencoder protection:
    Regular: 0.9384 --> 0.7909
  FGSM: 0.2437 --> 0.6678
  PGD: 0.0013 --> 0.7284

resnet56_cifar10 - time:  15.432801961898804 

  (Test Accuracy) With autoencoder protection:
    Regular: 0.9524 --> 0.7909
  FGSM: 0.4109 --> 0.7092
  PGD: 0.2537 --> 0.7476

resnet110_cifar10 - time:  31.615257501602173 

  (Test Accuracy) With autoencoder protection:
    Regular: 0.9625 --> 0.8033
  FGSM: 0.4526 --> 0.724
  PGD: 0.3116 --> 0.7655

preresnet20_cifar10 - time:  5.542132377624512 

  (Test Accuracy) With autoencoder protection:
    Regular: 0.9334 --> 0.7979
  FGSM: 0.3481 --> 0.7086
  PGD: 0.1648 --> 0.7515

preresnet56_cifar10 - time:  15.8715341091156 



KeyboardInterrupt: 